In [1]:
import time
import pandas as pd

from collections import defaultdict
from tqdm import tqdm
from gendas.engine import Gendas
from gendas.utils import flatten
from gendas.experimental import HG19Source

In [2]:
%%bash
cat data/gendas.conf


[cadd]
type = tabix
file = cadd_score.tsv.gz
header = CHR, POS, REF, ALT, RAW, PHRED
ctypes = str, int, str, str, float, float
sequence= CHR
begin = POS
end = POS

[variants]
type = tabix
file = variants.tsv.gz
header = CHR, POS, REF, ALT, SAMPLE
ctypes = str, int, str, str, str
sequence = CHR
begin = POS
end = POS

[tcga]
type = tabix
file = tcga.txt.gz
header = CHR, POS, REF, ALT
ctypes = str, int, str, str
sequence = CHR
begin = POS
end = POS


[exons]
type = tabix
file = cds_exons.tsv.gz
header = CHR, START, STOP, GENE
ctypes = str, int, int, str
sequence = CHR
begin = START
end = STOP
indices = GENE,

[genes]
type = tabix
file = cds_annotations.tsv.gz
header = CHR, GENE, SYMBOL, BEGIN, END, STRAND
ctypes = str, str, str, int, int, str
sequence = CHR
begin = BEGIN
end = END


In [3]:
# Create a Gendas engine
gd = Gendas('data/gendas.conf')
gd['hg19'] = HG19Source()

In [4]:
def mut_rank(gd, baserow):

    # Get scores of all the position in this gene group by tri>alt
    context = defaultdict(list)
    for r in gd['cadd'].merge(gd['hg19']):
        key = "{}>{}".format(r['hg19'][-1:1], r['cadd']['ALT'])
        context[key].append(r['cadd']['PHRED'])

    rows = []
    for m in gd['variants'].merge(gd['cadd'], on=['REF', 'ALT']).merge(gd['hg19']):

        key = "{}>{}".format(m['hg19'][-1:1], m['variants']['ALT'])
        ctx_scores = context[key]

        # Create the output row
        row = dict(baserow)
        for k, v in m['variants'].items():
            row[k] = v

        row['KEY'] = key
        row['SCORE'] = m['cadd']['PHRED']
        row['CONTEXT'] = ctx_scores

        rows.append(row)

    return rows

In [5]:
%%time
data = pd.DataFrame.from_dict(
    flatten(
        tqdm(gd.groupby(gd['exons']['GENE']).aggregate(mut_rank), total=len(set(gd['exons']['GENE'])))
    ), orient='columns'
)

100%|██████████| 242/242 [00:05<00:00, 42.01it/s]


CPU times: user 406 ms, sys: 27.4 ms, total: 434 ms
Wall time: 5.8 s


In [6]:
data[['GENE', 'CONTEXT', 'KEY']].head(1).to_dict(orient='index')

{0: {'GENE': 'ENSG00000154639', 'CONTEXT': [], 'KEY': 'TT>C'}}

In [7]:
len(data)

6085